<a href="https://colab.research.google.com/github/jinensetpal/evalita/blob/master/evaluation/ensemble_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Minor Changes to Evaluate Ensembles -**

In [2]:
! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/ensemble ./

In [3]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [4]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [5]:
def evaluate(model, X, y):
  kfold = StratifiedKFold(n_splits=5).split(X, y)
  scores = []
  for k, (train, test) in enumerate(kfold):
    score = model.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    scores.append(score[1] * 100)
    scoring((model(X[test]) > 0.5).numpy().astype("int32").tolist(), y[test])
  print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))

In [68]:
def scoring(predictions, y_test):
  fp = 0
  tp = 0
  fn = 0 
  c = 0
  incorrect = []

  for pred, test in zip(predictions, y_test):
    if pred[0] != test[0]:
      incorrect.append(c)
    if pred[0] == 1:
      if pred[0] == test[0]:
        tp += 1
      else:
        fp += 1
    elif pred[0] != test[0]:
      fn += 1
    c += 1

  print(tp, fp, fn)

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)
  print('Precision: ', precision, '\nRecall: ', recall, '\nF1 Score: ', f1)
  
  return precision, recall, f1

In [15]:
## The following are the images the ensemble got wrong; note it's shuffled from original!
def calc_inc(pred, y):
  inc = []
  c = 0
  for i in range(len(pred)):
    if pred[i][0] == y[i][0]:
      c += 1
    else:
      print(i, '-', pred[i][0], y[i][0])
      inc.append(i)
  print('\nAccuracy: ' + str(c / len(pred) * 100) + '%')
  return inc

In [64]:
def ensemble_calc(a, d, r):
  final = []
  for i in range(len(a)):
    if a[i][0] + d[i][0] + r[i][0] >= 2:
      final.append(1)
    else:
      final.append(0)
  final = np.array(final)
  final = final.reshape(final.shape[0], 1)
  return final

In [7]:
from tensorflow import keras

alex = keras.models.load_model('ensemble/alexnet_ffwd')
dense = keras.models.load_model('ensemble/densenet_ffwd')
resnet = keras.models.load_model('ensemble/resnet_ffwd')

In [8]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [9]:
import numpy as np
import pickle

des_X, alex_X, res_X, y = pickle.load(open('densenest_embeddings.pkl', 'rb')), pickle.load(open('alexnet_embeddings.pkl', 'rb')), np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [11]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ohe(create_map())
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = pickle.load(open('cls_umberto_embeddings.pkl', 'rb'))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

(1600, 1)
(1600, 1001)
(1600, 4097)
(1600, 2049)
(1600, 1)
(1600, 1002)
(1600, 4098)
(1600, 2050)
(1600, 71)
(1600, 1073)
(1600, 4169)
(1600, 2121)
(1600, 768)
(1600, 1841)
(1600, 4937)
(1600, 2889)


In [25]:
from sklearn.model_selection import train_test_split

res_X_train, res_X_test, res_y_train, res_y_test = train_test_split(res_X, y, test_size=0.2, stratify=y, random_state=42)
des_X_train, des_X_test, des_y_train, des_y_test = train_test_split(des_X, y, test_size=0.2, stratify=y, random_state=42)
alex_X_train, alex_X_test, alex_y_train, alex_y_test = train_test_split(alex_X, y, test_size=0.2, stratify=y, random_state=42)

In [41]:
### Cumulative Score

a = (alex(alex_X_test) > 0.5).numpy().astype("int32").tolist()
d = (dense(des_X_test) > 0.5).numpy().astype("int32").tolist()
r = (resnet(res_X_test) > 0.5).numpy().astype("int32").tolist()
final = ensemble_calc(a, d, r)

(320, 1)


In [27]:
from sklearn.metrics import classification_report

a_inc = calc_inc((alex(alex_X_test) > 0.5).numpy().astype("int32"), alex_y_test)
d_inc = calc_inc((dense(des_X_test) > 0.5).numpy().astype("int32"), alex_y_test)
r_inc = calc_inc((resnet(res_X_test) > 0.5).numpy().astype("int32"), alex_y_test)
f_inc = calc_inc(final, alex_y_test)

9 - 1 0
15 - 1 0
16 - 1 0
18 - 1 0
21 - 1 0
23 - 1 0
25 - 0 1
27 - 0 1
36 - 0 1
41 - 0 1
61 - 0 1
64 - 1 0
65 - 1 0
73 - 1 0
74 - 1 0
79 - 0 1
88 - 0 1
93 - 0 1
95 - 0 1
98 - 1 0
101 - 0 1
122 - 1 0
126 - 1 0
133 - 1 0
138 - 0 1
139 - 1 0
157 - 1 0
158 - 1 0
162 - 0 1
163 - 0 1
167 - 1 0
175 - 1 0
177 - 0 1
185 - 1 0
186 - 1 0
189 - 1 0
194 - 0 1
196 - 1 0
200 - 1 0
201 - 0 1
202 - 1 0
204 - 1 0
211 - 0 1
212 - 1 0
216 - 1 0
217 - 1 0
221 - 1 0
229 - 1 0
235 - 0 1
244 - 1 0
246 - 1 0
247 - 1 0
248 - 1 0
249 - 1 0
252 - 0 1
256 - 1 0
265 - 1 0
271 - 0 1
273 - 0 1
286 - 0 1
289 - 1 0
293 - 0 1
294 - 1 0
295 - 1 0
317 - 0 1

Accuracy: 79.6875%
9 - 1 0
11 - 1 0
15 - 1 0
33 - 0 1
54 - 0 1
61 - 0 1
64 - 1 0
65 - 1 0
74 - 1 0
77 - 1 0
79 - 0 1
82 - 0 1
84 - 1 0
94 - 1 0
95 - 0 1
102 - 1 0
103 - 0 1
122 - 1 0
123 - 1 0
124 - 0 1
125 - 0 1
126 - 1 0
131 - 1 0
133 - 1 0
138 - 0 1
145 - 0 1
157 - 1 0
162 - 0 1
169 - 1 0
176 - 0 1
180 - 1 0
184 - 1 0
185 - 1 0
186 - 1 0
187 - 1 0
198 - 0 1
202 - 1

In [28]:
tot = {}
c = 0

In [29]:
for i in a_inc + d_inc + r_inc:
  if i not in tot:
    tot[i] = 1
  else:
    tot[i] += 1

In [30]:
for i in tot:
  if tot[i] > 1:
    print(i, '-', tot[i])
    c += 1

9 - 2
15 - 3
16 - 2
25 - 2
27 - 2
36 - 2
61 - 3
64 - 3
65 - 2
74 - 2
79 - 3
88 - 2
93 - 2
95 - 2
122 - 2
126 - 2
133 - 2
138 - 2
157 - 3
162 - 2
175 - 2
185 - 2
186 - 3
194 - 2
196 - 2
200 - 2
201 - 2
202 - 3
212 - 2
221 - 2
246 - 2
248 - 3
252 - 3
256 - 2
265 - 2
271 - 3
293 - 2
317 - 3
33 - 2
84 - 2
94 - 2
103 - 2
124 - 2
131 - 2
145 - 2
198 - 2


In [31]:
print(c) ## no of incorrectly predicted files from testing dataset

46


In [32]:
## Suffle Dataframe

seed = np.random.randint(0, 100000)  

np.random.seed(seed)
np.random.shuffle(des_X)
np.random.seed(seed)
np.random.shuffle(alex_X)
np.random.seed(seed)
np.random.shuffle(res_X)
np.random.seed(seed)
np.random.shuffle(y)

In [49]:
## Individual Scores

import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

classification_report((alex(alex_X) > 0.5).numpy().astype("int32"), y)
classification_report((dense(des_X) > 0.5).numpy().astype("int32"), y)
classification_report((resnet(res_X) > 0.5).numpy().astype("int32"), y)

# evaluate(alex, alex_X, y)
# evaluate(dense, des_X, y)
# evaluate(resnet, res_X, y)

'              precision    recall  f1-score   support\n\n           0       0.92      0.83      0.88       892\n           1       0.81      0.91      0.86       708\n\n    accuracy                           0.87      1600\n   macro avg       0.87      0.87      0.87      1600\nweighted avg       0.87      0.87      0.87      1600\n'

```
AlexNet - 
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       799
           1       0.93      0.93      0.93       801

    accuracy                           0.93      1600
   macro avg       0.93      0.93      0.93      1600
weighted avg       0.93      0.93      0.93      1600

DenseNet - 
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       773
           1       0.91      0.87      0.89       827

    accuracy                           0.89      1600
   macro avg       0.89      0.89      0.89      1600
weighted avg       0.89      0.89      0.89      1600

Resnet -
              precision    recall  f1-score   support

           0       0.92      0.83      0.88       892
           1       0.81      0.91      0.86       708

    accuracy                           0.87      1600
   macro avg       0.87      0.87      0.87      1600
weighted avg       0.87      0.87      0.87      1600

Overall -
              precision    recall  f1-score   support

           0       0.84      0.87      0.86       157
           1       0.87      0.85      0.86       163

    accuracy                           0.86       320
   macro avg       0.86      0.86      0.86       320
weighted avg       0.86      0.86      0.86       320
```



In [77]:
kfold = StratifiedKFold(n_splits=5).split(alex_X_test, alex_y_test) ## Since dataframe numbers are only shifted, varied df calls can be made

pres = []
rec = []
f1 = []

for k, (train, test) in enumerate(kfold):
  a = (alex(alex_X_test[test]) > 0.5).numpy().astype("int32").tolist()
  d = (dense(des_X_test[test]) > 0.5).numpy().astype("int32").tolist()
  r = (resnet(res_X_test[test]) > 0.5).numpy().astype("int32").tolist()

  final = ensemble_calc(a, d, r)
  d_inc = calc_inc(final, alex_y_test[test])
  f_p, f_r, f_f1 = scoring(final, alex_y_test[test])
  pres.append(f_p)
  rec.append(f_r)
  f1.append(f_f1)
  print()

print("Precision: %.2f (+/- %.2f)" % (np.mean(pres), np.std(pres)))
print("Recall: %.2f (+/- %.2f)" % (np.mean(rec), np.std(rec)))
print("F1 Score: %.2f (+/- %.2f)" % (np.mean(f1), np.std(f1)))

9 - 1 0
15 - 1 0
16 - 1 0
25 - 0 1
27 - 0 1
33 - 0 1
36 - 0 1
61 - 0 1

Accuracy: 87.5%
27 3 5
Precision:  0.9 
Recall:  0.84375 
F1 Score:  0.870967741935484

1 - 1 0
2 - 1 0
10 - 1 0
15 - 0 1
20 - 1 0
24 - 0 1
29 - 0 1
30 - 1 0
31 - 0 1
39 - 0 1
58 - 1 0
60 - 1 0
62 - 1 0
63 - 1 0

Accuracy: 78.125%
27 9 5
Precision:  0.75 
Recall:  0.84375 
F1 Score:  0.7941176470588235

0 - 0 1
10 - 0 1
17 - 0 1
29 - 1 0
34 - 0 1
47 - 1 0
57 - 1 0
58 - 1 0

Accuracy: 87.5%
28 4 4
Precision:  0.875 
Recall:  0.875 
F1 Score:  0.875

2 - 0 1
4 - 1 0
6 - 0 1
8 - 1 0
9 - 0 1
10 - 1 0
20 - 1 0
29 - 1 0
54 - 1 0
56 - 1 0
60 - 0 1

Accuracy: 82.8125%
28 7 4
Precision:  0.8 
Recall:  0.875 
F1 Score:  0.8358208955223881

1 - 1 0
9 - 1 0
15 - 0 1
37 - 0 1
61 - 0 1

Accuracy: 92.1875%
28 2 3
Precision:  0.9333333333333333 
Recall:  0.9032258064516129 
F1 Score:  0.9180327868852459

Precision: 0.85 (+/- 0.07)
Recall: 0.87 (+/- 0.02)
F1 Score: 0.86 (+/- 0.04)


In [35]:
### Cumulative Score

a = (alex(alex_X) > 0.5).numpy().astype("int32").tolist()
d = (dense(des_X) > 0.5).numpy().astype("int32").tolist()
r = (resnet(res_X) > 0.5).numpy().astype("int32").tolist()

final = []
for i in range(len(a)):
  # print(a[i][0] + d[i][0] + r[i][0])
  if a[i][0] + d[i][0] + r[i][0] >= 2:
    final.append(1)
  else:
    final.append(0)
final = np.array(final)
final = final.reshape(final.shape[0], 1)
print(final.shape)

(1600, 1)


In [36]:
scoring(final, y) # For total data; data leaked & deprecated!!

734 49 62
Precision:  0.9374201787994891 
Recall:  0.9221105527638191 
F1 Score:  0.9297023432552248


In [ ]:
X_inc = []

for i, val in enumerate(X):
  for j in incorrect:
    if np.all(X_test[j][:1000] == val[:1000]):
      X_inc.append(i)

In [ ]:
print(X_inc) # Incorrectly classified representative to dataset

[22, 125, 138, 256, 260, 265, 294, 311, 332, 417, 450, 456, 462, 489, 502, 504, 520, 533, 536, 545, 566, 589, 609, 613, 622, 629, 652, 700, 708, 742, 803, 840, 851, 868, 891, 911, 993, 1018, 1058, 1071, 1072, 1104, 1121, 1163, 1172, 1219, 1224, 1240, 1266, 1336, 1339, 1357, 1373, 1403, 1448, 1470, 1472, 1509, 1513, 1519, 1588, 1598]
